In [ ]:
%load_ext watermark


In [ ]:
from IPython.display import display
import pandas as pd
import polars as pl
from teeplot import teeplot as tp
from tqdm import tqdm

from pylib.viz._size_fixation_areaplot import size_fixation_areaplot


In [ ]:
%watermark -diwmuv -iv


In [ ]:
teeplot_subdir = "cupy-5050-traits"
teeplot_subdir


In [ ]:
data = [
    {
        "url": "https://osf.io/duam2/download",
        "population structure": "well-mixed"
    },
    {
        "url": "https://osf.io/8q5v6/download",
        "population structure": "2D demes",
    },
    # {
    #     "url": "https://osf.io/s67d2/download",
    #     "population structure": "1D demes",
    # },
]
dfs = []
for item in tqdm(data):
    df = pd.read_parquet(item["url"])
    df["population structure"] = item["population structure"]
    df["available beneficial mutations"] = df["nBen"]
    df = pl.from_pandas(df)
    df = (
        df.with_columns(
            pl.when(pl.col("nRowSubgrid") == 0)
            .then(pl.col("nRow"))
            .otherwise(pl.col("nRowSubgrid"))
            .alias("nRowSubgrid"),
            pl.when(pl.col("nColSubgrid") == 0)
            .then(pl.col("nCol"))
            .otherwise(pl.col("nColSubgrid"))
            .alias("nColSubgrid"),
        )
        .with_columns(
            (pl.col("row").cast(pl.UInt64) // pl.col("nRowSubgrid")).alias(
                "rowGroup"
            ),
            (pl.col("col").cast(pl.UInt64) // pl.col("nColSubgrid")).alias(
                "colGroup"
            ),
        )
        .with_columns(
            (
                pl.col("colGroup")
                + (pl.col("colGroup").max() + 1) * pl.col("rowGroup")
            ).alias("group"),
        )
        .with_columns(
            (
                pl.col("trait value").replace_strict(
                    {0: "normomutator", 1: "hypermutator"},
                )
            ).alias("genotype"),
        )
        .with_columns(
            (
                pl.col("nColSubgrid").cast(pl.UInt64)
                * pl.col("nRowSubgrid").cast(pl.UInt64)
                * pl.col("tilePopSize")
            ).alias("population size"),
        )
        .with_columns(
            (pl.col("trait count") / pl.col("tilePopSize")).alias(
                "fixation probability"
            ),
        )
    )
    assert (df["nCol"] > 0).all()
    assert (df["nRow"] > 0).all()
    assert (df["nColSubgrid"] > 0).all()
    assert (df["nRowSubgrid"] > 0).all()
    assert (df["tilePopSize"] > 0).all()
    assert (df["population size"] > 0).all()
    dfs.append(
        df.group_by(
            [
                "group",
                "genotype",
                "replicate",
                "population size",
                "population structure",
                "available beneficial mutations",
            ],
        ).agg(
            pl.col("fixation probability").mean(),
        ),
    )


In [ ]:
dfxs = pl.concat(dfs, how="vertical_relaxed").to_pandas()
dfxs["fixation\nprobability"] = dfxs["fixation probability"]
dfxs["population\nstructure"] = dfxs["population structure"]
dfxs["available\nbeneficial\nmutations"] = dfxs["available beneficial mutations"]

display(dfxs.describe())
display(dfxs.head())
display(dfxs.tail())


In [ ]:
apn = int(dfxs["population size"].min())  # agents per node
apn


In [ ]:
for errorbar in "sd", "se", "ci", None:
    tp.tee(
        size_fixation_areaplot,
        fixprobs_df=dfxs[
            dfxs["available beneficial mutations"] <= 4
        ],
        x="population size",
        y="fixation probability",
        hue="genotype",
        hue_order=["normomutator", "hypermutator"],
        col="available\nbeneficial\nmutations",
        row="population\nstructure",
        style="genotype",
        style_order=["normomutator", "hypermutator"],
        errorbar=errorbar,
        height=2,
        facet_kws=dict(margin_titles=True),
        markers=True,
        teeplot_outexclude="post",
        teeplot_subdir=teeplot_subdir,
        teeplot_postprocess=f"""
teed.figure.suptitle("n={apn} agents per node", x=0.63, y=1.06)
teed.set_titles(col_template="{{col_var}} = {{col_name:.0f}}")
teed.set_titles(row_template="{{row_name}}")
teed.set_ylabels("fixation probability")
sns.move_legend(
    teed,
    "lower center",
    bbox_to_anchor=(0.25, 0.95),
    frameon=False,
    ncol=2,
    title=None,
)
""",
    )
